In [1]:
!pip install cohere
!pip install annoy
import cohere

import numpy as np

import warnings

warnings.filterwarnings('ignore')

from annoy import AnnoyIndex

import numpy as np

import pandas as pd

Defaulting to user installation because normal site-packages is not writeable

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


   -------------------------------------- 150.3/150.3 kB 898.0 kB/s eta 0:00:00
   -------------------------------------- 546.3/546.3 kB 817.2 kB/s eta 0:00:00
   -------------------------------------- 121.1/121.1 kB 886.6 kB/s eta 0:00:00



  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.10.2 requires ruamel-yaml, which is not installed.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.
tensorboard 2.11.0 requires protobuf<4,>=3.9.2, but you have protobuf 4.25.3 which is incompatible.
tensorflow-intel 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.25.3 which is incompatible.
botocore 1.24.32 requires urllib3<1.27,>=1.25.4, but you have urllib3 2.2.1 which is incompatible.


Defaulting to user installation because normal site-packages is not writeable
     ------------------------------------ 647.5/647.5 kB 948.2 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for annoy: filename=annoy-1.17.3-cp39-cp39-win_amd64.whl size=53093 sha256=1f7ece73a511b91d9c4f43262860d14a18bb0e801459edb0ef1f4811f422be25
  Stored in directory: c:\users\pankaj pant\appdata\local\pip\cache\wheels\09\a9\54\37478e65995fe712f7da465749da9ddb21db6b1a599d591ac7
Successfully built annoy


DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
#question = "What about your education?"
text = ""
with open('story.txt') as f:
    text = f.read()

In [4]:
texts = text.split('\n\n')
# Clean up to remove empty spaces and new lines
texts = np.array([t.strip(' \n') for t in texts if t])

co = cohere.Client('yF0ryoncYMB6rlHQZybJNn9urUVfO1yx38svAuP9')

# Get the embeddings
response = co.embed(

    texts=texts.tolist(),

).embeddings

In [5]:
# Check the dimensions of the embeddings
embeds = np.array(response)
# Create the search index, pass the size of embedding
search_index = AnnoyIndex(embeds.shape[1], 'angular')

# Add all the vectors to the search index

for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])

search_index.build(10) # 10 trees
search_index.save('test.ann')

True

In [6]:
def search_text(query):
    # Get the query's embedding
    query_embed = co.embed(texts=[query]).embeddings
    # Retrieve the nearest neighbors
    similar_item_ids = search_index.get_nns_by_vector(query_embed[0],
                                                    10,
                                                  include_distances=True)

    search_results = texts[similar_item_ids[0]]
    return search_results

In [7]:
def ask_llm(question, num_generations=1):
    # Search the text archive
    results = search_text(question)
    # Get the top result
    context = results[0]
    # Prepare the prompt
    prompt = f"""
    More information about Australian beaches at australia.com:
    {context}
    Question: {question}
    Extract the answer of the question from the text provided.
    If the text doesn't contain the answer,

    reply that the answer is not available."""

    prediction = co.generate(
        prompt=prompt,
        max_tokens=70,
        model="command-nightly",
        temperature=0.5,
        num_generations=num_generations
    )

    return prediction.generations


In [11]:
question = "what cocodile want"
results = ask_llm(question)
print(results[0].text)

The crocodile wanted to eat the monkey's heart.


In [9]:
!pip install pyttsx3
!pip install speechrecognition
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install pyaudio

Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
'apt' is not recognized as an internal or external command,
operable program or batch file.


Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [14]:
import speech_recognition as sr
import pyttsx3
# Initialize the recognizer
r = sr.Recognizer()

# Function to convert text to
# speech
def SpeakText(command):
    # Initialize the engine
    engine = pyttsx3.init()
    engine.say(command)
    engine.runAndWait()



while(1):
    try:
        print("------speak-----")
        with sr.Microphone() as source2:
            r.adjust_for_ambient_noise(source2, duration=0.2)
            audio2 = r.listen(source2)

            # Using google to recognize audio
            question = r.recognize_google(audio2)
            question = question.lower()
            if question=="exit":
                break
            results = ask_llm(question)
            answer = results[0].text

            print("Request  : ",question)
            print("Response : ",answer)

            SpeakText(answer)

    except sr.RequestError as e:
        print("Could not request results; {0}".format(e))

    except sr.UnknownValueError:
        print("unknown error occurred")

------speak-----
unknown error occurred
------speak-----
unknown error occurred
------speak-----
unknown error occurred
------speak-----
unknown error occurred
------speak-----
Request  :  where crocodile live
Response :  The crocodile lived in the river Ganges.
------speak-----
Request  :  what crocodile wife want
Response :  The crocodile's wife wants to eat the monkey's heart.
------speak-----
Request  :  where monkey live
Response :  The monkey lived in a big jamun tree.
------speak-----
Request  :  where is the jamun tree
Response :  The jamun tree is on the bank of the river Ganges.
------speak-----
Request  :  what is the moral of story
Response :  The moral of the story is that one should not be naive and trust others too easily, as even friends can have ulterior motives and deceive you.
------speak-----
Request  :  who is sourav
Response :  The answer is not available.
------speak-----
Request  :  correct exit
Response :  The answer is not available.
------speak-----
